In [ ]:
import argparse
import logging
from config import Config
from data.data_loader import DataLoader
from data.preprocessor import Preprocessor
from model.llm import LLM
from model.rag import RAG
from training.trainer import Trainer
from evaluation.evaluator import Evaluator
from utils.logger import setup_logger
from inference.tutor import Tutor

def parse_args():
    parser = argparse.ArgumentParser(description="RAG-Enhanced LLM Tutor")
    parser.add_argument("--mode", choices=["train", "evaluate", "infer"], required=True)
    parser.add_argument("--config", default="config.yaml", help="Path to config file")
    return parser.parse_args()

def main():
    args = parse_args()
    config = Config(args.config)
    setup_logger(config.logging.level, config.logging.file)
    logger = logging.getLogger(__name__)

    data_loader = DataLoader(config.data)
    preprocessor = Preprocessor(config.preprocessing)

    llm = LLM(config.model.llm)
    rag = RAG(config.model.rag, llm)

    if args.mode == "train":
        trainer = Trainer(config.training, rag)
        train_data = data_loader.load_train_data()
        preprocessed_data = preprocessor.preprocess(train_data)
        trainer.train(preprocessed_data)
    elif args.mode == "evaluate":
        evaluator = Evaluator(config.evaluation)
        eval_data = data_loader.load_eval_data()
        preprocessed_data = preprocessor.preprocess(eval_data)
        results = evaluator.evaluate(rag, preprocessed_data)
        logger.info(f"Evaluation results: {results}")
    elif args.mode == "infer":
        tutor = Tutor(rag, config.inference)
        tutor.start_interactive_session()

if __name__ == "__main__":
    main()